### Overall
PSHA --> Deagg --> Common sources --> Hazard


#### Deagg:
==> Get Deagg ==> Add to DB ==> 

#### Common:
==> Create Views ==> Create Common sources view ==> Check Validity ==> Update common source view

#### Hazard:
==> Compute Parameters ==> Get Hazard

# Imports

In [6]:
import json
import urllib
import time
import sqlite3
import pandas as pd
import math
from IPython.display import clear_output
import ipynb.fs.defs.pyExcel


# Function Definitions


## Deaggregation:

In [14]:
def getInput():
    locations = []
    
    ans = "Y"
    while(ans != "N"):
        print("\nEnter location details: ")   
        entry = {}
        entry["loc_id"] = input("loc_id (for example GGB for Golden Gate Bridge) : ")
        entry["name"] = input("name: ")
        entry["latitude"] = float(input("latitude (-110 for 110° S): "))
        entry["longitude"] = float(input("longitude (-110 for 110° W): "))
        imt = int(input("Intensity Measure (1 for PGA, 2 for SA 0.2, 3 for SA 1.0): "))
        
        imt_list = ["PGA", "SA0P2", "SA1P0"]
        entry["imt"] = imt_list[imt - 1]
        locations.append(entry)
        
        ans = "a"
        while (ans != "Y" and ans != "N"):
            ans = input("Enter another entry? (Y/N):")
        clear_output()
        print("Entered locations: ")
        for location in locations:
            print(location)
    return locations

# ------------------------------------------------------------------------------------------------------------------------------
    
def getDeaggData(lat, long, imt = "PGA", rp = 2475, vs30 = 760, region = 'WUS'):
    url = f"https://earthquake.usgs.gov/nshmp-haz-ws/deagg?edition=E2014&region={region}&longitude={long}&latitude={lat}&imt={imt}&vs30={vs30}&returnperiod={rp}"
    deagg = urllib.request.urlopen(url)
    return json.load(deagg)

# ------------------------------------------------------------------------------------------------------------------------------

def getAllDeaggData(lat, long, imt, region = 'WUS', vs30 = 760):
    """
        Returns a dictionary of deaggregation data 
        for all combinations of return period and Intensity measure
        for a given latitude and logitude
        
        Dictionary key example: data['475_PGA']
    """
    
    rp_list = [475, 975, 2475]
    
    sources_data = {}
    sources_name = {}
    
    for rp in rp_list:
        key = f"{rp}_{imt}"
        extract = getDeaggData(lat, long, imt, rp)

        # Sources corresponding to the component: Total (data[0])
        sources = extract['response'][0]['data'][0]['sources']
        sources_data[key] = sources   
    
    return sources_data
  

# ------------------------------------------------------------------------------------------------------------------------------    
            
def splitKey(key):
    loc = key.find('_')
    return_period = key[:loc]
    imt = key[loc+1:]
    return imt, return_period

# ------------------------------------------------------------------------------------------------------------------------------    

def modifySourceName(name):
    mod_name = name
    if name.find('[') != -1:
        mod_name = name[0:name.find('[')]
    
    return mod_name

# ------------------------------------------------------------------------------------------------------------------------------    

def createTable(db_path):
    db = sqlite3.connect(db_path, timeout = 10)
    cur = db.cursor()
    
    cur.execute(
    """
        CREATE TABLE IF NOT EXISTS Location(
                        loc_id TEXT PRIMARY KEY,
                        name TEXT,
                        latitude FLOAT,
                        longitude FLOAT,
                        imt TEXT
                        )
    """
    )
    
    cur.execute(
    """
        CREATE TABLE IF NOT EXISTS DeaggData(
                    loc_id TEXT,
                    return_period TEXT,
                    imt TEXT,
                    source_set TEXT,
                    source_name TEXT,
                    mod_source_name TEXT,
                    r FLOAT,
                    m FLOAT,
                    eps FLOAT,
                    latitude FLOAT,
                    longitude FLOAT,
                    azimuth FLOAT,
                    contribution FLOAT,
                    PRIMARY KEY (loc_id, return_period, imt, source_set, source_name)
                    )
    """ 
    )
    
    cur.execute(
    """
        CREATE TABLE IF NOT EXISTS Deagg_w_hazard(
                    loc_id TEXT,
                    return_period TEXT,
                    imt TEXT,
                    source_set TEXT,
                    source_name TEXT,
                    mod_source_name TEXT,
                    r FLOAT,
                    m FLOAT,
                    eps FLOAT,
                    latitude FLOAT,
                    longitude FLOAT,
                    azimuth FLOAT,
                    contribution FLOAT,
                    hazard FLOAT,
                    SA1P0 FLOAT,
                    SA0P3 FLOAT,
                    PRIMARY KEY (loc_id, return_period, imt, source_set, source_name)
                    )
    """ 
    )
    
    db.commit()
    cur.close()
    db.close()
    return

# ------------------------------------------------------------------------------------------------------------------------------    

def insert(cur, table_name, data_dict):
    attribute = []
    value = []
    for key in data_dict:
        attribute.append(key)
        value.append(data_dict[key])
    try:
        cur.execute(
            f"""
                INSERT INTO {table_name} {tuple([att for att in attribute])}
                VALUES {tuple([val for val in value])}
            """
            )
    except sqlite3.IntegrityError:
        print("The following record already exists: \n", value)
    except:
        print("An error occured when inserting following record: \n", value)
    return

# ------------------------------------------------------------------------------------------------------------------------------    

def addSourceData(cur, loc_id, sources):
    entry = {}
    entry["loc_id"] = loc_id
    for key in sources:
        for i in range(len(sources[key])):
            if sources[key][i]['type'] == 'SET':
                set_name = sources[key][i]['name']
            

            if sources[key][i]['type'] == "SINGLE":
                imt, rp = splitKey(key)
                entry["imt"] = imt
                entry["return_period"] = rp
                entry["source_set"] = set_name
                entry["source_name"] = sources[key][i]['name']
                entry["mod_source_name"] = modifySourceName(sources[key][i]['name'])
                entry["contribution"] = sources[key][i]['contribution']
                entry["r"] = sources[key][i]['r']
                entry["m"] = sources[key][i]['m']
#                 entry["eps"] = sources[key][i]['ε']
                entry["eps"] = sources[key][i]['\u03B5']
                entry["azimuth"] = sources[key][i]['azimuth']
                entry["latitude"] = sources[key][i]['latitude']
                entry["longitude"] = sources[key][i]['longitude']
                
                

                insert(cur, "DeaggData", entry)
                
            
    return

# ------------------------------------------------------------------------------------------------------------------------------    



## Common Sources:

In [11]:
def createLocDeaggView(db, loc_id):
    cur = db.cursor()
    cur.execute(f"""CREATE VIEW IF NOT EXISTS {loc_id} AS
                 SELECT * FROM DeaggData
                 WHERE loc_id is '{loc_id}';""")
    cur.close()
    db.commit()
    return

# ------------------------------------------------------------------------------------------------------------------------------    

def createCommonSourcesTable(db, locations):  
    try:
        cur = db.cursor()
#         cur.execute("DROP VIEW CommonSources")
        cur.execute("DELETE FROM CommonSources;")
        cur.execute("DROP TABLE CommonSources;")
        cur.close()
    except:
        cur.close()
        
#     query = "CREATE VIEW CommonSources AS "
    query = "CREATE TABLE CommonSources AS "
    for location in locations:
        loc_id = location["loc_id"]
        query = query + f"SELECT DISTINCT return_period, source_set, mod_source_name FROM {loc_id} INTERSECT "
    query = query[:-11] + ";"
    cur = db.cursor()
    cur.execute(query)
    db.commit()
    cur.close()
    return

# ------------------------------------------------------------------------------------------------------------------------------    

def maxSeparation(coordinates_list):
    p1 = coordinates_list[0]
    p2 = coordinates_list[1]
    maxSep = distance(p1, p2)
    
    i = 2
    while (i < len(coordinates_list)):
        p3 = coordinates_list[i]
        dist1 = distance(p1, p3)
        dist2 = distance(p2, p3)
        
        if dist1 > maxSep:
            maxSep = dist1
            p1 = p3
        
        if dist2 > maxSep:
            maxSep = dist2
            p2 = p3
        
        i += 1
    return maxSep

# ------------------------------------------------------------------------------------------------------------------------------    

def distance(p1, p2):
    R = 6371 # km
    lat1, lon1 = p1
    lat2, lon2 = p2
    
    lat_r1 = lat1 * math.pi/180 #convert to radians
    lat_r2 = lat2 * math.pi/180
    del_lat = (lat2-lat1) * math.pi/180
    del_lon = (lon2-lon1) * math.pi/180

    a = math.sin(del_lat/2) * math.sin(del_lat/2) + \
          math.cos(lat_r1) * math.cos(lat_r2) * \
          math.sin(del_lon/2) * math.sin(del_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c # in km
    return d

# ------------------------------------------------------------------------------------------------------------------------------    

def getSRL(M):
    '''
        Returns SRL in km (M between 5.6 and 8.1)
    '''
    a = -3.55
    b = 0.74

    sa = 0.37
    sb = 0.05
    SRL = 10**(a + b*M)
    return SRL

# ------------------------------------------------------------------------------------------------------------------------------    

def filterSources(db):    
    cur = db.cursor()
    cur.execute("SELECT * FROM CommonSources")
    com_sources = cur.fetchall()
    
    for source in com_sources:
        rp = source[0]
        source_set = source[1]
        mod_source_name = source[2]
        
        cur.execute(f"""
                    SELECT latitude, longitude FROM DeaggData 
                    WHERE return_period = {rp} AND source_set IS "{source_set}" AND
                    mod_source_name IS "{mod_source_name}";
                    """)
        coordinates_list = cur.fetchall()
        max_sep = maxSeparation(coordinates_list)

        cur.execute(f"""
                    SELECT MIN(m) FROM DeaggData 
                    WHERE return_period = {rp} AND source_set IS "{source_set}" AND
                    mod_source_name IS "{mod_source_name}";
                    """)
        
        min_m = cur.fetchall()        
        SRL = getSRL(min_m[0][0])
        
        if max_sep > SRL:
            cur.execute(f"""DELETE FROM CommonSources
                            WHERE return_period = {rp} AND source_set IS "{source_set}" AND
                            mod_source_name IS "{mod_source_name}"; """)
    cur.close()
    db.commit()
    return 

# ------------------------------------------------------------------------------------------------------------------------------    

def entriesOfInterest(db):   
    try:
        cur = db.cursor()
#         cur.execute("DROP VIEW EntryOfInterest")
        cur.execute("DELETE FROM EntryOfInterest;")
        cur.execute("DROP TABLE EntryOfInterest;")
        cur.close()
    except:
        cur.close()
    
    cur = db.cursor()
    cur.execute(f"""CREATE TABLE EntryOfInterest AS
                 SELECT * FROM DeaggData 
                 WHERE (return_period, source_set, mod_source_name) IN CommonSources""")
    
    cur.execute(f"""ALTER TABLE EntryOfInterest 
                    ADD COLUMN hazard FLOAT NULL;
                    """)
    cur.execute(f"""ALTER TABLE EntryOfInterest 
                    ADD COLUMN SA0P3 FLOAT NULL;
                    """)
    cur.execute(f"""ALTER TABLE EntryOfInterest 
                    ADD COLUMN SA1P0 FLOAT NULL;
                    """)
    
    
    db.commit()
    cur.close()
    return

# ------------------------------------------------------------------------------------------------------------------------------    


## Wrapper Function

In [12]:
def execute(locations, db_path):
    # Create table if not already there:
    createTable(db_path)
    
    # Default Site Class - B
    vs30_ = 760

# -------------------------------------------------------------------------------------------------------------------------   
# --------------------------------------------------- Deaggregation Data ---------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------- 
    db = sqlite3.connect(db_path, timeout = 10)
    for location in locations:
        lat = location["latitude"]
        long = location["longitude"]
        loc_id = location["loc_id"]
        imt = location["imt"]

        # Check if data already exist
        cur = db.cursor()
        query = f"""SELECT * FROM Location WHERE latitude IS {lat} AND longitude IS {long} AND imt IS '{imt}';"""
        cur.execute(query)
        entries = cur.fetchall()
        cur.close()

        # If data is not added, add data
        if len(entries) == 0:
            # Add location to db
            cur = db.cursor()
            insert(cur, "Location", location)
            cur.close()
            
            # obtaine Deagg sources data in the form of dictionary of json 
            sources_data = getAllDeaggData(lat, long, imt, vs30 = vs30_)
            
            # Add deagg details in the database
            cur = db.cursor()
            addSourceData(cur, loc_id, sources_data)
            db.commit()
            cur.close()
            

# -------------------------------------------------------------------------------------------------------------------------   
# ----------------------------------------------------- Common Sources-----------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------- 
    # Create Location Views 
    for location in locations:
        loc_id = location["loc_id"]
        createLocDeaggView(db, loc_id) 
    
    # Create Common Sources Table
    createCommonSourcesTable(db, locations)

    # Filter Common sources according to SRL
    filterSources(db)

    # Create final Entries of Interest table 
    entriesOfInterest(db)
    db.close()
    
    
# -------------------------------------------------------------------------------------------------------------------------   
# --------------------------------------------------------- Hazard ---------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------- 

    ipynb.fs.defs.pyExcel.executeHazardAlgorithm(db_path, vs30_)

    return